In [1]:
import os

In [2]:
%pwd

'g:\\Python Codes\\kidney-disease-detection-mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'g:\\Python Codes\\kidney-disease-detection-mlops'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config 

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(
                f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-27 01:43:44,623: INFO: common: yaml file: G:\Python Codes\kidney-disease-detection-mlops\config\config.yaml loaded successfully]
[2025-10-27 01:43:44,631: INFO: common: yaml file: G:\Python Codes\kidney-disease-detection-mlops\params.yaml loaded successfully]
[2025-10-27 01:43:44,634: INFO: common: created directory at: artifacts]
[2025-10-27 01:43:44,636: INFO: common: created directory at: artifacts/data_ingestion]


[2025-10-27 01:43:44,639: INFO: 1199208304: Downloading data from https://drive.google.com/file/d/172xRLsitipzymBi9EslZ2u2VZN3OJwIz/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=172xRLsitipzymBi9EslZ2u2VZN3OJwIz
From (redirected): https://drive.google.com/uc?export=download&id=172xRLsitipzymBi9EslZ2u2VZN3OJwIz&confirm=t&uuid=05944313-eb2e-4947-a5bb-dc91ec5120ac
To: g:\Python Codes\kidney-disease-detection-mlops\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [05:25<00:00, 2.89MB/s] 

[2025-10-27 01:49:14,577: INFO: 1199208304: Downloaded data from https://drive.google.com/file/d/172xRLsitipzymBi9EslZ2u2VZN3OJwIz/view?usp=sharing into artifacts/data_ingestion/data.zip]
